<a href="https://colab.research.google.com/github/ankit-kothari/H1B-Data-Analysis/blob/master/H1B_Data_Analysis_Using_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import plotly
from plotly import graph_objs as go
from plotly.graph_objs import *
from flask import Flask
import pandas as pd
import numpy as np
import os
import sqlite3
import copy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
conn = sqlite3.connect("/Users/ankitkothari/Documents/COMPLETED_PROJECTS/H1B_data_analysis/us_h1b.db")

## Sum of Initial Approvals, Initial Denials, Continuing Approvals, Continuing Denials by Employer



*   Only Employers with more than 100 H1B applications are counted using Having condition



In [ ]:
h1b_query = ''' 
select  
Employer, SUM(Initial_Approvals) AS Initial_Approvals, 
SUM(Initial_Denials) AS Initial_Denials, 
SUM(Continuing_Approvals) AS Continuing_Approvals, 
SUM(Continuing_Denials) AS Continuing_Denials, 
NAICS
from h1b
group by Employer
having SUM(Initial_Approvals)>100;'''


df = pd.read_sql_query(h1b_query, conn)
print(df.iloc[0:10])

Employer  Initial_Approvals  Initial_Denials  Continuing_Approvals  Continuing_Denials  NAICS
0                     3I INFOTECH INC                408               62                  1119               119.0     54
1      8K MILES SOFTWARE SERVICES INC                115               32                   170                60.0     54
2                     A T KEARNEY INC                119                6                   118                 3.0     54
3                    A10 NETWORKS INC                108                1                   117                 3.0     54
4          A2Z DEVELOPMENT CENTER INC                117                4                   207                 2.0     54
5  A2Z DEVELOPMENT CTR INC DBA LAB126                103                2                   341                 5.0     54
6                          A9 COM INC                128                2                   238                 1.0     54
7                             ABB INC        

## SUM of (Initial Approvals + Continuing Approvals) and (Initial Denials + Continuing Denials) by Employer 
 
*   Query ordered by Max Denials



In [ ]:
h1b_query2 = ''' 
select  
Employer,  SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS Approvals, 
SUM(Initial_Denials) + SUM(Continuing_Denials) AS Denials
from h1b
where Fiscal_Year !='2019'
group by Employer
order by 3 desc;'''


df2 = pd.read_sql_query(h1b_query2, conn)
print(df2)


                               Employer  Approvals  Denials
0                             COGNIZANT     143770  11049.0
1                                  TATA      75435   3932.0
2                               INFOSYS      97961   3638.0
3                                 WIPRO      60012   3531.0
4                              DELOITTE      34842   3053.0
...                                 ...        ...      ...
265144         ZYXEL COMMUNICATIONS INC          7      0.0
265145                        ZYZYX INC          4      0.0
265146  ZZ CAPITAL INTERNATIONAL US LLC          1      0.0
265147          ZZ INVESTMENT GROUP INC          3      0.0
265148              [X+1] SOLUTIONS INC          1      0.0

[265149 rows x 3 columns]


## Approval (Initial Approvals + Continuing Approvals)  and  Denials (Initial Denials + Continuing Denials) from Fiscal Year 2009 to 2018 for Amazon

In [19]:
h1b_query3a = ''' 
select  
h1b.Employer,h1b.Fiscal_Year,  SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS Approvals, 
SUM(h1b.Initial_Denials) + SUM(h1b.Continuing_Denials) AS Denials
from h1b LEFT JOIN (
    select  
       Employer, 
       Fiscal_Year, 
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS Approvals_2016, 
       SUM(Initial_Denials) + SUM(Continuing_Denials) AS Denials_2016
       from h1b
       where Fiscal_Year !='2019'and Employer LIKE 'AMAZON'
       group by Fiscal_Year

) h1b2 ON h1b.Employer = h1b2.Employer
where h1b.Fiscal_Year !='2019' and h1b.Employer LIKE 'AMAZON'
group by h1b.Fiscal_Year
;
'''

df3a = pd.read_sql_query(h1b_query3a, conn)
print(df3a)

  Employer  Fiscal_Year  Approvals  Denials
0   AMAZON         2009       4620       30
1   AMAZON         2010       5240       20
2   AMAZON         2011       8880       20
3   AMAZON         2012      11980       60
4   AMAZON         2013      16650      110
5   AMAZON         2014      19860      180
6   AMAZON         2015      21940      230
7   AMAZON         2016      32200      390
8   AMAZON         2017      54130      530
9   AMAZON         2018      62690     1660


## Comparing Denial Rates pre and post 2016 by State Top(15)

In [22]:
h1b_query8= ''' 
with h1b_table_by_state AS
(
select  
h1b.State,   
SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS approvals_pre_2016, 
SUM(h1b.Initial_Denials) + SUM(h1b.Continuing_Denials) AS denials_pre_2016,
(CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)) / (CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)+CAST(SUM(h1b.Initial_Approvals) AS REAL) + CAST(SUM(h1b.Continuing_Approvals) AS REAL))*100 AS denial_pre_2016,
h1b2.State,
h1b2.approvals_post_2016, 
h1b2.denials_post_2016,
h1b2.denial_post_2016

from h1b LEFT JOIN (
    select  
       State,  
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS approvals_post_2016, 
       SUM(Initial_Denials) + SUM(Continuing_Denials) AS denials_post_2016,
       (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)) / (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)+CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL))*100 AS denial_post_2016,
       Fiscal_Year
       from h1b
       where Fiscal_Year !='2019' and Fiscal_Year>2016
       group by State

) h1b2 ON h1b.State = h1b2.State
where h1b.Fiscal_Year !='2019' and h1b.Fiscal_Year<=2016
group by h1b.State
)
select 
hs.State,
hs.denial_pre_2016 AS denial_rate_pre_2016,
hs.denial_post_2016 AS denial_rate_post_2016,
hs.denial_post_2016 - hs.denial_pre_2016 AS delta
from h1b_table_by_state hs 
order by 4 desc
;
'''

df8 = pd.read_sql_query(h1b_query8, conn)
df8.set_index('State', inplace=True)
df8 = df8.applymap(lambda x: round(x,2))

print(f' df8 \n {df8}')

 df8 
        denial_rate_pre_2016  denial_rate_post_2016  delta
State                                                    
NM                     4.26                  15.74  11.48
PR                     6.66                  17.99  11.33
GA                     6.72                  17.13  10.41
IL                     5.85                  14.13   8.27
NJ                     7.94                  15.93   8.00
IA                     4.90                  12.85   7.94
VA                     7.60                  14.98   7.39
FL                     7.50                  14.86   7.36
SC                     4.79                  11.93   7.14
MI                     6.77                  13.45   6.69
OH                     6.13                  12.17   6.04
TX                     4.42                  10.24   5.82
NC                     4.54                  10.05   5.51
MO                     4.37                   9.78   5.41
CT                     3.66                   8.97   5.31
NH     

## EMPLOYER WITH HIGHEST PERCENTAGE SHARE OF H1B BY STATE


*   Using LEFT join to find the list of Employers by State
*   Using Max aggregate function identifying the Maximum 



In [ ]:
h1b_query9= '''
with h1b_table_by_state AS
(
select  
h1b.State AS State,
h1b.Employer AS Employer,
SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS total_visas_State, 
Max(h1b2.approvals) AS max_visa_in_state
from h1b LEFT JOIN (
    select  
       State, Employer,
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS approvals
       from h1b
       where Fiscal_Year !='2019'
       group by State,Employer
       order by 3 desc

) h1b2 ON h1b.State = h1b2.State and h1b.Employer = h1b2.Employer 
where h1b.Fiscal_Year !='2019'
group by h1b.State
order by 4 desc
)
select
htbs.State,
htbs.Employer,
htbs.total_visas_State,
htbs.max_visa_in_state,
(CAST(htbs.max_visa_in_state AS REAL)/ CAST(htbs.total_visas_State AS REAL))*100 AS percent_visa
from h1b_table_by_state htbs 
order by 5 desc;
'''

df9 = pd.read_sql_query(h1b_query9, conn)
print(f' df 9 \n {df9}')

 df 9 
    State                           Employer  total_visas_State  max_visa_in_state  percent_visa
0     MD                               TATA             121999              74504     61.069353
1     VT                       ITECH US INC               2994               1677     56.012024
2     NC                                IBM              63728              30357     47.635262
3     ID              MICRON TECHNOLOGY INC               3914               1791     45.758815
4     AR                           WAL-MART              10463               4120     39.376852
5     WA                          MICROSOFT              95411              36151     37.889761
6     TN                            CUMMINS              24813               6773     27.296175
7     PA                           DELOITTE             109917              28968     26.354431
8     RI                       CVS/PHARMACY               7072               1679     23.741516
9     TX                        

## Comparing Denial Rates pre and post 2016 by Employer Top(15)

In [ ]:
h1b_query11= ''' 
with h1b_table_by_state AS
(
select  
h1b.Employer,   
SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS approvals_pre_2016, 
SUM(h1b.Initial_Denials) + SUM(h1b.Continuing_Denials) AS denials_pre_2016,
(CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)) / (CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)+CAST(SUM(h1b.Initial_Approvals) AS REAL) + CAST(SUM(h1b.Continuing_Approvals) AS REAL))*100 AS denial_pre_2016,
h1b2.Employer,
h1b2.approvals_post_2016, 
h1b2.denials_post_2016,
h1b2.denial_post_2016

from h1b LEFT JOIN (
    select  
       Employer,  
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS approvals_post_2016, 
       SUM(Initial_Denials) + SUM(Continuing_Denials) AS denials_post_2016,
       (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)) / (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)+CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL))*100 AS denial_post_2016,
       Fiscal_Year
       from h1b
       where Fiscal_Year !='2019' and Fiscal_Year>2016
       group by Employer

) h1b2 ON h1b.Employer = h1b2.Employer
where h1b.Fiscal_Year !='2019' and h1b.Fiscal_Year<=2016
group by h1b.Employer
having approvals_pre_2016 > 2000 and h1b2.approvals_post_2016>500
)
select 
hs.Employer,
hs.denial_pre_2016 AS denial_rate_pre_2016,
hs.denial_post_2016 AS denial_rate_post_2016,
hs.denial_post_2016 - hs.denial_pre_2016 AS delta_denial_rates_pre_post2016
from h1b_table_by_state hs 
order by 4 desc
limit 15
;
'''

df11 = pd.read_sql_query(h1b_query11, conn)
print(f' df11 \n {df11}')

 df11 
                     Employer  denial_rate_pre_2016  denial_rate_post_2016  delta_denial_rates_pre_post2016
0                  CAPGEMINI              4.463745              25.078370                        20.614625
1                 KFORCE INC             10.397554              24.423963                        14.026410
2   CSC COVANSYS CORPORATION              6.000000              17.551963                        11.551963
3             UST GLOBAL INC              3.282602              13.593156                        10.310554
4                     SYNTEL              3.130160              11.523929                         8.393769
5     PRICEWATERHOUSECOOPERS              1.937185               9.343545                         7.406360
6                   MINDTREE              3.115424              10.402999                         7.287575
7                  ACCENTURE              2.282917               8.929167                         6.646250
8                        HCL 

## Total Approvals By State and Percentage Share of H1B Visas

In [ ]:
h1b_query13 = ''' 
with visas_by_state AS
(
select  
SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS sum_IA 
from h1b
where Fiscal_Year !='2019'
)
select  
State, SUM(Initial_Approvals)  + SUM(Continuing_Approvals) total_visa,
(CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL)) / (CAST(vs.sum_IA AS REAL))*100 AS percentage_total_visa
from h1b, visas_by_state vs
where Fiscal_Year !='2019'
group by State
order by 2 desc
;
'''

df13 = pd.read_sql_query(h1b_query13, conn)
print(f' df13 {df13.iloc[0:10]}')

df13   State  total_visa  percentage_total_visa
0    CA      496311              16.872466
1    TX      411275              13.981603
2    NJ      384639              13.076092
3    NY      228969               7.783973
4    IL      160014               5.439796
5    MD      121999               4.147448
6    MA      119515               4.063002
7    PA      109917               3.736711
8    MI       99444               3.380674
9    WA       95411               3.243569


## Total Approvals By Employer and Percentage Share of H1B Visas

In [ ]:
h1b_query14 = ''' 
with visas_by_Employer AS
(
select  
SUM(Initial_Approvals) AS sum_IA 
from h1b
where Fiscal_Year !='2019'
)
select  
Employer,
(CAST(SUM(Initial_Approvals) AS REAL) / CAST(vs.sum_IA AS REAL))*100 AS percent_total_H1B 
from h1b, visas_by_Employer vs
where Fiscal_Year !='2019'
group by Employer
order by 2 desc
limit 10
;
'''

df14 = pd.read_sql_query(h1b_query14, conn)
print(f' df14 {df14}')

 df14     Employer  percent_total_H1B
0  COGNIZANT           3.843342
1       TATA           3.051281
2    INFOSYS           2.830154
3      WIPRO           2.140800
4  ACCENTURE           1.666490
5        IBM           1.210432
6  MICROSOFT           1.196447
7     AMAZON           1.033530
8   DELOITTE           0.978589
9        HCL           0.919743


## Comparing Denial Rate Between US Companies and Indian Companies pre and post 2016

1.   Using self join to calcualte denial rates pre and post 2016
2.   Using CASE Statement tagging Indian and US Companies



In [ ]:
h1b_query16= ''' 
with h1b_table_by_state AS
(
select  
h1b.Employer,   
SUM(h1b.Initial_Approvals)  + SUM(h1b.Continuing_Approvals) AS approvals_pre_2016, 
SUM(h1b.Initial_Denials) + SUM(h1b.Continuing_Denials) AS denials_pre_2016,
(CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)) / (CAST(SUM(h1b.Initial_Denials) AS REAL) + CAST(SUM(h1b.Continuing_Denials) AS REAL)+CAST(SUM(h1b.Initial_Approvals) AS REAL) + CAST(SUM(h1b.Continuing_Approvals) AS REAL))*100 AS denial_pre_2016,
h1b2.Employer,
h1b2.approvals_post_2016, 
h1b2.denials_post_2016,
h1b2.denial_post_2016

from h1b LEFT JOIN (
    select  
       Employer,  
       SUM(Initial_Approvals)  + SUM(Continuing_Approvals) AS approvals_post_2016, 
       SUM(Initial_Denials) + SUM(Continuing_Denials) AS denials_post_2016,
       (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)) / (CAST(SUM(Initial_Denials) AS REAL) + CAST(SUM(Continuing_Denials) AS REAL)+CAST(SUM(Initial_Approvals) AS REAL) + CAST(SUM(Continuing_Approvals) AS REAL))*100 AS denial_post_2016,
       Fiscal_Year
       from h1b
       where Fiscal_Year !='2019' and Fiscal_Year>2016
       group by Employer

) h1b2 ON h1b.Employer = h1b2.Employer
where h1b.Fiscal_Year !='2019' and h1b.Fiscal_Year<=2016
group by h1b.Employer
having approvals_pre_2016 > 2000 and h1b2.approvals_post_2016>500
)
select 
hs.Employer,
CASE WHEN hs.Employer IN ("APPLE","FACEBOOK","AMAZON","MICROSOFT","GOOGLE") THEN "US COMPANIES" ELSE "INDIAN COMPANIES" END AS COMPANY_ORIGIN,
hs.denial_pre_2016 AS denial_rate_pre_2016,
hs.denial_post_2016 AS denial_rate_post_2016,
hs.denial_post_2016 - hs.denial_pre_2016 AS delta_denial_rates_pre_post2016
from h1b_table_by_state hs 
where hs.Employer IN ("APPLE","FACEBOOK","AMAZON","MICROSOFT","GOOGLE", "TATA", "ACCENTURE", "WIPRO","CAPGEMINI","MINDTREE")
order by 4 desc
limit 15
;
'''

df16 = pd.read_sql_query(h1b_query16, conn)
print(f' df16 {df16}')

df16     Employer    COMPANY_ORIGIN  denial_rate_pre_2016  denial_rate_post_2016  delta_denial_rates_pre_post2016
0  CAPGEMINI  INDIAN COMPANIES              4.463745              25.078370                        20.614625
1   MINDTREE  INDIAN COMPANIES              3.115424              10.402999                         7.287575
2      WIPRO  INDIAN COMPANIES              4.671449               9.664537                         4.993088
3  ACCENTURE  INDIAN COMPANIES              2.282917               8.929167                         6.646250
4       TATA  INDIAN COMPANIES              5.155401               4.522951                        -0.632450
5     AMAZON      US COMPANIES              0.849604               1.840181                         0.990578
6  MICROSOFT      US COMPANIES              1.032591               1.719986                         0.687395
7     GOOGLE      US COMPANIES              0.400352               1.348564                         0.948212
8      APPLE  

## Top 3 Counties By State with Highest Denial Rate


1.   Using the View with a sum of Denials and Total Applications
2.   Using the Dense Rank Function, Partition by State and Ranking Order by Denial Rate



In [ ]:
h1b_query15 = ''' 
with visas_by_County AS
(
select  
State,
City,
SUM(Initial_Denials) + SUM(Continuing_Denials) AS denials_by_city,
SUM(Initial_Denials) + SUM(Continuing_Denials)+SUM(Initial_Approvals)+ SUM(Continuing_Approvals) AS total_applications_by_city 
from h1b
where Fiscal_Year !='2019'
group by State,City
having (SUM(Initial_Denials) + SUM(Continuing_Denials))>10
)
select
rate_by_county.State,
rate_by_county.City,
rate_by_county.denial_rate,
rate_by_county.denial_rate_rank
from 
(
select
vs.State,
vs.City,
vs.total_applications_by_city,
vs.denials_by_city,
(CAST(vs.denials_by_city AS REAL) / CAST(vs.total_applications_by_city AS REAL))*100 denial_rate,
DENSE_RANK () OVER ( 
			PARTITION BY State
			ORDER BY ((CAST(vs.denials_by_city AS REAL) / CAST(vs.total_applications_by_city AS REAL))*100) DESC
		) denial_rate_rank 
from visas_by_County vs
) rate_by_county 
where rate_by_county.denial_rate_rank <= 3 and rate_by_county.State IN ("VA","CA")

;
'''


df15 = pd.read_sql_query(h1b_query15, conn)
print(f' df15 {df15}')


df15   State          City  denial_rate  denial_rate_rank
0    CA    CARMICHAEL    51.401869                 1
1    CA       ARTESIA    39.325843                 2
2    CA   WESTMINSTER    38.958333                 3
3    VA       CLIFTON    29.850746                 1
4    VA       ASHBURN    25.133043                 2
5    VA  SOUTH RIDING    22.033898                 3
